**<h1>Zdjęcia Aut i Kotów</h1>**

<h1>Wstęp</h1>
Pamiętasz Marka Adamczyka, który prosił cię o pomoc w algorytmie znajdującym twórcę tweeta? Okazało się że ten pomysł to był strzał w dziesiątkę! Niestety przed wyjechaniem w podróż dookoła świata zapomniał ci on dać twoich należności za wykonaną pracę. Gdy po wielu miesiącach udało ci się z nim skontaktować, wspomniał ci on, że wczoraj zauważył, że zdjęcia samochodów, kotów, oraz innych fascynujących rzeczy, które zrobił są na tyle niewyraźne, że algorytm w jego telefonie nie jest w stanie ich automatycznie przypisać, i że przez będzie zmuszony do potwórzenia tej podróży. Niestety nie jest on ich w stanie sam przyporządkować ponieważ podczas tej podróży udało mu się zrobić tych zdjęć około... 60000. Nie masz pojęcia, jakim sposobem był on w stanie zrobić tyle zdjęć, lecz wiesz, że jeśli wybierze się w kolejną podróż, to raczej już nie zobaczysz swoich pieniędzy. Postanowiłeś więc zaproponować mu, iż napiszesz program, który automatycznie przyporządkuje zdjęcia do klas, których automat w jego telefonie nie był w stanie rozwiązać. Wywiąż się z obietnicy, jeśli wynik będzie zadowalający, to być może zobaczysz twoje należne pieniądze

# <h2>Dostarczone pliki</h2>

- `train_labeled.csv` - Dane treningowe z etykietami
- `train_unlabeled.csv` - Dane treningowe bez etykiet
- `test.csv` - Dane testowe
- `przykodp.csv` - Przykładowy plik w formacie w jakim ma być `odpowiedzi.csv`.
- `Zadanie.ipynb` - Notebook startowy do pracy nad modelem.

# <h2>Foldery z obrazami</h2>

  -  `/train_labeled` - Zawiera obrazy odpowiadające etykietom z train_labeled.csv.
  -  `/train_unlabeled` - Zawiera obrazy odpowiadające danym z train_unlabeled.csv (bez etykiet).
  -  `/test` - Zawiera obrazy do testowania.
# <h2>Twoje zadanie</h2>

### **1. Self-supervised Pretraining**
- Zbuduj własną architekturę modelu.
- Naucz model reprezentacji na danych `train_unlabeled.csv` za pomocą self-supervised learning. Możesz użyć metod takich jak:
  - Contrastive learning (np. SimCLR).
  - Autoenkodery.

### **2. Fine-tuning**
- Użyj wytrenowanego modelu do klasyfikacji na danych z częściowymi etykietami (`train_labeled.csv`).
- Dostosuj hiperparametry (np. liczba epok, batch size, learning rate) dla swojego modelu.

### **3. Ewaluacja**
- Przeprowadź ewaluację swojego modelu na zbiorze testowym (`test.csv`).

# <h2>Ograniczenia</h2>

- Czas działania twojego kodu(trening i ewaluacja) na T4 na Google Colab powinien wynosić maksymalnie 10 minut.
- Nie wolno korzystać z uprzednio wytrenowanych modeli oraz ze zbiorów danych innych niż dostarczony.
- Twój kod może trenować się tylko i wyłącznie na danych z pliku train.csv.
- Wszystkie dopuszczalne biblioteki są dostępne w pliku requirements.txt

# <h2>Ocenanie</h2>

Ocena zależy od wartości accuracy w sposób liniowy, przy czym:

- Jeśli `accuracy < 0.5`, to liczba punktów wynosi 0.
- Jeśli `accuracy >= 0.7`, to liczba punktów wynosi 1.
- Dla wartości `accuracy` pomiędzy 0.5 a 0.7, liczba punktów rośnie liniowo.

Wzór na obliczenie punktów (P):

$$ P = \frac{accuracy - 0.5}{0.2} $$

# <h2>Ograniczenia</h2>

1. Kod może korzystać wyłącznie z danych dostarczonych w plikach `train_labeled.csv`, `train_unlabeled.csv` oraz `test.csv`.
2. Czas treningu i ewaluacji na GPU T4 w Google Colab powinien wynosić maksymalnie 10 minut.
3. Użytkownik musi samodzielnie zbudować model od podstaw, bez korzystania z gotowych modeli (np. ResNet, VGG).

<h2>Rozwiązanie</h2>

- W tym zadaniu musisz musisz dołączyć plik Zadanie.ipynb, który po włączeniu utworzy plik odpowiedzi.csv gdzie będą znajdowały sie odpowiedzi, w formacie takim jak przykodp.csv, oraz plik odpowiedzi.csv.



In [69]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms
import pandas as pd
from PIL import Image
import os
import csv

device = "cuda" if torch.cuda.is_available() else "cpu"

print(f"using: {device}")


using: cuda


In [82]:
class ImageDataset(Dataset):
    def __init__(self, csv_path, transform=None, labeled=True, test=False):
        """
        Args:
            csv_path (str): Path to the CSV file.
            transform (callable, optional): A function/transform to apply to the images.
            labeled (bool): Whether the dataset includes labels.
        """
        self.data = pd.read_csv(csv_path)
        self.transform = transform
        self.labeled = labeled
        self.test = test

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 0]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        if self.labeled:
            if self.test:
                return image, img_path
            
            label = self.data.iloc[idx, 1]
            return image, label
        else:
            return image
    
def load_datasets(val_size=0):
    """
    Funkcja ładująca dane z plików CSV i przygotowująca DataLoadery.
    
    TODO:
    1. Wczytaj ścieżki do plików CSV dla zbiorów danych (np. train_labeled.csv, train_unlabeled.csv, test.csv).
    2. Zdefiniuj odpowiednie transformacje dla obrazów (np. augmentacja, normalizacja).
    3. Utwórz obiekty ImageDataset dla każdego z wymienionych zbiorów danych:
       - Zbiór treningowy oznaczony (labeled).
       - Zbiór treningowy nieoznaczony (unlabeled).
       - Zbiór testowy.
    4. Zainicjalizuj DataLoadery dla każdego zbioru, ustawiając odpowiednie parametry, takie jak:
       - `batch_size`
       - `shuffle` (dla zbioru treningowego).
       - `num_workers` (dla optymalizacji wydajności).
    5. Upewnij się, że DataLoadery zwracają dane w odpowiednim formacie (np. (image, label) dla labeled, tylko image dla unlabeled).
    6. Przetestuj poprawność działania DataLoaderów na przykładzie małego batcha danych.
    
    Zwróć:
    - `train_labeled_loader`: DataLoader dla oznaczonego zbioru treningowego.
    - `train_unlabeled_loader`: DataLoader dla nieoznaczonego zbioru treningowego.
    - `test_loader`: DataLoader dla zbioru testowego.
    """
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    train_labeled = ImageDataset("./train_labeled.csv", transform=transform, labeled=True)
    train_unlabeled = ImageDataset("./train_unlabeled.csv", transform=transform, labeled=False)
    test = ImageDataset("./test.csv", transform=transform, labeled=True, test=True)
    
    train_size = int((1-val_size) * len(train_labeled))  # 80% for training
    val_size = len(train_labeled) - train_size  # Remaining 20% for validation
    
    if val_size > 0:
        train_dataset, val_dataset = random_split(train_labeled, [train_size, val_size])
    
        train_labeled_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
        val_labeled_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
    else:
        train_labeled_loader = DataLoader(train_labeled, batch_size=32, shuffle=True)
        val_labeled_loader = None
        
    train_unlabeled_loader = DataLoader(train_unlabeled, batch_size=64, shuffle=True)
    test_loader = DataLoader(test, batch_size=32)
    
    return train_labeled_loader, val_labeled_loader, train_unlabeled_loader, test_loader


In [113]:
class SuperDuperProDINO(nn.Module):
    def __init__(self):
        super(SuperDuperProDINO, self).__init__()
        self.stage = "pretraining"
        
        self.encoder = torch.nn.Sequential(
            torch.nn.Conv2d(3, 16, (3,3)),
            torch.nn.Dropout(0.3),
            torch.nn.ReLU(),
            torch.nn.AvgPool2d((2,2)),
            torch.nn.Conv2d(16, 24, (3,3)),
            torch.nn.Dropout(0.3),
            torch.nn.Flatten(),
            torch.nn.ReLU(),
            torch.nn.Linear(4056, 50)
        )
        
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(50, 4056),
            torch.nn.ReLU(),
            torch.nn.Unflatten(1, (24, 13, 13)),  # Matches output of encoder's Flatten input
            torch.nn.Dropout(0.3),
            torch.nn.ConvTranspose2d(24, 16, kernel_size=3, stride=1, padding=1),  # Output: (16, 13, 13)
            torch.nn.Upsample(scale_factor=2, mode='nearest'),  # Upsamples to (16, 26, 26)
            torch.nn.ReLU(),
            torch.nn.Dropout(0.3),
            torch.nn.ConvTranspose2d(16, 8, kernel_size=3, stride=1, padding=1),  # Output: (8, 26, 26)
            torch.nn.Upsample(size=(32, 32), mode='nearest'),  # Upsamples directly to (8, 32, 32)
            torch.nn.ReLU(),
            torch.nn.ConvTranspose2d(8, 3, kernel_size=3, stride=1, padding=1),  # Output: (3, 32, 32)
            torch.nn.Sigmoid()  # Output activation for pixel values [0, 1]
        )
        
        self.classification_head = torch.nn.Sequential(
            torch.nn.Linear(50,10)
        )

    def forward(self, x):
        if self.stage == "pretraining":
            x = self.encoder(x)
            x = self.decoder(x)
            
            return x
        
        elif self.stage == "finetuning" or self.stage == "testing":
            x = self.encoder(x)
            x = self.classification_head(x)
            
            return x
        

In [115]:

def self_supervised_pretraining(unlabeled_loader, model, epochs=100, lr=0.01):
    """
    Pretraining modelu w trybie self-supervised.

    Args:
        unlabeled_loader (DataLoader): Dane bez etykiet.
        model (nn.Module): Model do pretrenowania.
        epochs (int): Liczba epok pretrenowania.
        lr (float): Współczynnik uczenia.

    TODO:
    1. Zaimplementuj metodę self-supervised:
       - Możesz użyć autoenkodera, kontrastowego uczenia (SimCLR, BYOL), itp.
    2. Ustaw odpowiednią funkcję straty i optymalizator.
    3. Przeprowadź trening i monitoruj utratę (loss).
    """
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    model.stage = "pretraining"
    
    for epoch in range(epochs):
        running_loss = []
        for imgs in unlabeled_loader:
            imgs = imgs.to(device)
            optimizer.zero_grad()
            
            otpt = model(imgs)
            
            loss = criterion(otpt, imgs)
            loss.backward()
            
            running_loss.append(loss.item())
            
            optimizer.step()
        print(f"Epoch {epoch+1}|{epochs}: Train Loss: {sum(running_loss)/len(running_loss)}")
        running_loss = []
            
    

In [116]:
def fine_tune_model(labeled_loader, model, epochs, lr):
    """
    Fine-tuning modelu na danych z etykietami.

    Args:
        labeled_loader (DataLoader): Dane z etykietami.
        model (nn.Module): Model do trenowania.
        epochs (int): Liczba epok treningu.
        lr (float): Współczynnik uczenia.

    TODO:
    1. Dodaj warstwę klasyfikacyjną do modelu.
    2. Ustaw funkcję straty (np. CrossEntropyLoss) i optymalizator.
    3. Wytrenuj model na zbiorze labeled.
    """
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    
    model.stage = "finetuning"
    
    for epoch in range(epochs):
        running_loss = []
        for imgs, labels in labeled_loader:
            imgs = imgs.to(device)
            optimizer.zero_grad()
            
            otpt = model(imgs)
            
            loss = criterion(otpt, labels.to(device))
            loss.backward()
            
            running_loss.append(loss.item())
            
            optimizer.step()
        print(f"Epoch {epoch+1}|{epochs}: Train Loss: {sum(running_loss)/len(running_loss)}")
        running_loss = []

In [117]:
def test_model(test_loader, model):
    model.stage = "testing"
    with torch.no_grad():
        score = 0
        max_score = 0
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            
            otpt = model(imgs)
            
            preds = torch.argmax(otpt, dim=1)
            
            score += torch.sum(preds==labels.to(device))
            max_score += labels.shape[0]
            
    print(f"Wynik to: {score}/{max_score} ({score/max_score})")
                      

In [118]:

def save_predictions(test_loader, model, output_file="odpowiedzi.csv"):
    """
    Generuje predykcje na danych testowych i zapisuje je do pliku CSV.
    
    Args:
        test_loader (DataLoader): Dane testowe bez etykiet.
        model (nn.Module): Wytrenowany model.
        output_file (str): Nazwa pliku wyjściowego (domyślnie 'predictions.csv').
    
    TODO:
    1. Wykonaj predykcje dla wszystkich obrazów w test_loader.
    2. Zapisz ścieżki do obrazów oraz przewidywane etykiety w formacie CSV.
       - Format CSV: 
         | image_path | label |
    """
    
    model.eval()
    model.stage = "testing"
    
    predictions = []

    with torch.no_grad():
        for imgs, imgs_paths in test_loader:
            imgs = imgs.to(device)
            
            otpt = model(imgs)
            
            preds = torch.argmax(otpt, dim=1).cpu().tolist()

            for path, label in zip(imgs_paths, preds):
                predictions.append((path, label))
    
    # Zapis wyników do pliku CSV
    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["image_path", "label"])  # Nagłówek pliku CSV
        writer.writerows(predictions)
            
            
            
    print(f"Zapisano predykcje do pliku csv!")


In [ ]:
def main():
    labeled_loader, _, unlabeled_loader, test_loader = load_datasets()

    model = SuperDuperProDINO().to(device)

    print("Rozpoczęcie pretreningu...")
    self_supervised_pretraining(unlabeled_loader=unlabeled_loader, model=model, epochs=3, lr=0.001)

    print("Rozpoczęcie fine-tuningu...")
    fine_tune_model(labeled_loader=labeled_loader, model=model, epochs=50, lr=0.001)

    print("Rozpoczęto predykcję klas ze zbioru testowego...")
    save_predictions(test_loader=test_loader, model=model)


if __name__ == "__main__":
    main()

Rozpoczęcie pretreningu...


KeyboardInterrupt: 

In [119]:
labeled_loader, val_loader, unlabeled_loader, test_loader = load_datasets(0.1)

model = SuperDuperProDINO().to(device)

print("Rozpoczęcie pretreningu...")
self_supervised_pretraining(unlabeled_loader=unlabeled_loader, model=model, epochs=5, lr=0.001)

print("Rozpoczęcie fine-tuningu...")
fine_tune_model(labeled_loader=labeled_loader, model=model, epochs=30, lr=0.001)

print("Rozpoczęto predykcję klas ze zbioru testowego...")
test_model(test_loader=val_loader, model=model)

# print("Generowanie predykcji na danych testowych...")
# save_predictions(test_loader, model, output_file="odpowiedzi.csv")


Rozpoczęcie pretreningu...
Epoch 1|5: Train Loss: 0.02215034877668319
Epoch 2|5: Train Loss: 0.013482788211664794
Epoch 3|5: Train Loss: 0.01221939077422845
Epoch 4|5: Train Loss: 0.011748988325500793
Epoch 5|5: Train Loss: 0.01153499772834866
Rozpoczęcie fine-tuningu...
Epoch 1|30: Train Loss: 2.1086151969264932
Epoch 2|30: Train Loss: 1.841505658458656
Epoch 3|30: Train Loss: 1.674167319082878
Epoch 4|30: Train Loss: 1.5346079493912173
Epoch 5|30: Train Loss: 1.3970847297722184
Epoch 6|30: Train Loss: 1.331325972583932
Epoch 7|30: Train Loss: 1.2507357639326175
Epoch 8|30: Train Loss: 1.1613928348245754
Epoch 9|30: Train Loss: 1.1221788189780544
Epoch 10|30: Train Loss: 1.0713389792912442
Epoch 11|30: Train Loss: 1.0327525911196855
Epoch 12|30: Train Loss: 0.9841045351095603
Epoch 13|30: Train Loss: 0.9389350011315144
Epoch 14|30: Train Loss: 0.8925288025761994
Epoch 15|30: Train Loss: 0.8441313929121259
Epoch 16|30: Train Loss: 0.7969137803769447
Epoch 17|30: Train Loss: 0.807282760

In [120]:
print("Rozpoczęto predykcję klas ze zbioru testowego...")
test_model(test_loader=labeled_loader, model=model)

Rozpoczęto predykcję klas ze zbioru testowego...
Wynik to: 1959/2250 (0.8706666827201843)
